In [ ]:
'''sh
pip install -r requirements.txt
cd code
'''


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
SEED = 42
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

### 第一问
银行根据中小微企业的实力、信誉对其信贷风险做出评估，然后依据信贷风险等因素来确定是否放贷及贷款额度、利率和期限等信贷策略。

某银行对确定要放贷企业的贷款额度为10~100万元；年利率为4%~15%；贷款期限为1年。附件1~3分别给出了123家有信贷记录企业的相关数据、302家无信贷记录企业的相关数据和贷款利率与客户流失率关系的2019年统计数据。

对附件1中123家企业的信贷风险进行量化分析，给出该银行在年度信贷总额固定时对这些企业的信贷策略。

##### 数据处理

##### 特征工程

In [2]:
import load

data = load.re_agg(load.get_data(num=123)).drop(['refund_in','refund_out'],axis = 1).fillna(1e-8)


##### 数据集整理

In [3]:
input_length = 12
label_length = 1
total_length = 36
batch_size = 123
input,test,label = load.get_tesors(data,input_length = input_length,label_length = label_length,total_length = total_length,batch_size = batch_size)

##### 训练神经网络模型，预测2020年的信用等级和违约概率

In [4]:
CONV_WIDTH = 3
OUT_STEPS = 1
num_labels = label.shape[-1]
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),

    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_labels,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_labels])
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=4, restore_best_weights=True,
                                                    mode='min')

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

multi_conv_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = multi_conv_model.fit(
    x=input, y = label, 
    batch_size=123, 
    epochs=100, 
    verbose=1, 
    callbacks=[early_stopping], 
    validation_split=0.3,  
    shuffle=True, 
    class_weight=None, 
    sample_weight=None, 
    initial_epoch=0, 
    steps_per_epoch=None, 
    validation_steps=None)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.9826 - mean_absolute_error: 0.5756 - val_loss: 3.3432 - val_mean_absolute_error: 1.4310
Epoch 2/100
1/1 [==============================] - 0s 23ms/step - loss: 0.9790 - mean_absolute_error: 0.5803 - val_loss: 3.3356 - val_mean_absolute_error: 1.4288
Epoch 3/100
1/1 [==============================] - 0s 23ms/step - loss: 0.9743 - mean_absolute_error: 0.5779 - val_loss: 3.3274 - val_mean_absolute_error: 1.4267
Epoch 4/100
1/1 [==============================] - 0s 23ms/step - loss: 0.9696 - mean_absolute_error: 0.5778 - val_loss: 3.3181 - val_mean_absolute_error: 1.4247
Epoch 5/100
1/1 [==============================] - 0s 23ms/step - loss: 0.9645 - mean_absolute_error: 0.5770 - val_loss: 3.3073 - val_mean_absolute_error: 1.4226
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 0.9587 - mean_absolute_error: 0.5759 - val_loss: 3.2950 - val_mean_absolute_error: 1.4203
Epoch 7/100
1/1 [=============

In [5]:
lstm_model = tf.keras.models.Sequential([
  #tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]),
  tf.keras.layers.LSTM(
    units = 4,
    activation="tanh",
    recurrent_activation="sigmoid",return_sequences= True),
    tf.keras.layers.LSTM(
    units = 16,
    activation="tanh",
    recurrent_activation="sigmoid",return_sequences= True),
  tf.keras.layers.LSTM(
    units = 32,
    activation="tanh",
    recurrent_activation="sigmoid",return_sequences= True),
    tf.keras.layers.LSTM(
    units = 64,
    activation="tanh",
    recurrent_activation="sigmoid",return_sequences= True),
    tf.keras.layers.LSTM(
    units = 128,
    activation="tanh",
    recurrent_activation="sigmoid"),
  tf.keras.layers.Dense(2),
  tf.keras.layers.Lambda(lambda x: x * 1000.0)
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=10, restore_best_weights=True,
                                                    mode='min')

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 0.1 * 10**(-epoch / 20))

lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
history = lstm_model.fit(
    x=input, y = label, 
    batch_size=25, 
    epochs=100, 
    verbose=1, 
    callbacks=[early_stopping], 
    validation_split=0.3,  
    shuffle=True, 
    class_weight=None, 
    sample_weight=None, 
    initial_epoch=0, 
    steps_per_epoch=None, 
    validation_steps=None)

Epoch 1/100
4/4 [==============================] - 11s 649ms/step - loss: 2.2885 - mean_absolute_error: 1.1993 - val_loss: 3.5965 - val_mean_absolute_error: 1.5239
Epoch 2/100
4/4 [==============================] - 0s 50ms/step - loss: 1.4763 - mean_absolute_error: 1.0045 - val_loss: 0.9695 - val_mean_absolute_error: 0.7863
Epoch 3/100
4/4 [==============================] - 0s 43ms/step - loss: 0.8648 - mean_absolute_error: 0.7286 - val_loss: 1.9654 - val_mean_absolute_error: 1.0543
Epoch 4/100
4/4 [==============================] - 0s 41ms/step - loss: 0.6946 - mean_absolute_error: 0.6881 - val_loss: 1.7026 - val_mean_absolute_error: 1.0640
Epoch 5/100
4/4 [==============================] - 0s 38ms/step - loss: 0.6028 - mean_absolute_error: 0.6004 - val_loss: 1.2050 - val_mean_absolute_error: 0.8513
Epoch 6/100
4/4 [==============================] - 0s 33ms/step - loss: 0.5191 - mean_absolute_error: 0.5148 - val_loss: 1.9071 - val_mean_absolute_error: 1.0397
Epoch 7/100
4/4 [=========

In [6]:
import tfm

tfm_model = tfm.tsf_model(
    input_shape = input.shape[1:],
    head_size=64,
    num_heads=8,
    ff_dim=16,
    num_transformer_blocks=4,
    mlp_units=[123],
    #mlp_dropout=4,
    #dropout=2,
)

tfm_model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()]
)
#model.summary()

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=5,restore_best_weights=True,
                                                    mode='min')]

history = tfm_model.fit(
    x = input,
    y = label,
    validation_split=0.3,
    epochs=200,
    batch_size=8,
    callbacks=callbacks,
)

Epoch 1/200
11/11 [==============================] - 4s 81ms/step - loss: 0.7796 - mean_absolute_error: 0.5361 - val_loss: 2.1480 - val_mean_absolute_error: 1.1479
Epoch 2/200
11/11 [==============================] - 0s 25ms/step - loss: 0.4512 - mean_absolute_error: 0.4102 - val_loss: 1.2514 - val_mean_absolute_error: 0.9077
Epoch 3/200
11/11 [==============================] - 0s 22ms/step - loss: 0.4382 - mean_absolute_error: 0.3893 - val_loss: 1.5774 - val_mean_absolute_error: 0.9870
Epoch 4/200
11/11 [==============================] - 0s 24ms/step - loss: 0.3962 - mean_absolute_error: 0.3707 - val_loss: 1.4411 - val_mean_absolute_error: 0.9567
Epoch 5/200
11/11 [==============================] - 0s 23ms/step - loss: 0.4076 - mean_absolute_error: 0.3859 - val_loss: 1.4477 - val_mean_absolute_error: 0.9587
Epoch 6/200
11/11 [==============================] - 0s 22ms/step - loss: 0.4162 - mean_absolute_error: 0.3795 - val_loss: 1.6528 - val_mean_absolute_error: 1.0115
Epoch 7/200
11/1

##### 测试结果

In [7]:
cnn_result = load.get_output(test,model = multi_conv_model)
cnn_result.to_csv('./results/cnn.csv')

lstm_result = load.get_output(test,model = lstm_model)
lstm_result.to_csv('./results/lstm.csv')

tfm_result = load.get_output(test,model = tfm_model)
tfm_result.to_csv('./results/tfm.csv')

##### 预测后302家企业的信用评级与违约概率

In [10]:
import load

data = load.re_agg(load.get_data(num=302)).drop(['refund_in','refund_out'],axis = 1).fillna(1e-8)
input_length = 12
label_length = 1
total_length = 36
batch_size = 302
input,test,label = load.get_tesors(data,input_length,label_length,total_length,batch_size)

In [11]:
cnn_result = load.get_output(test,model = multi_conv_model)
cnn_result.to_csv('./results/cnn.csv')

lstm_result = load.get_output(test,model = lstm_model)
lstm_result.to_csv('./results/lstm.csv')

tfm_result = load.get_output(test,model = tfm_model)
tfm_result.to_csv('./results/tfm.csv')

##### 确定信贷策略